<a href="https://colab.research.google.com/github/Chamara-g/pre-datastorm-v1-0/blob/master/pre_data_storm_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd

In [73]:
#read train data set
df = pd.read_csv("drive/My Drive/FYP/dataset/credit_card_default_train.csv", index_col=0)
df.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,NEXT_MONTH_DEFAULT
Client_ID,,,,,,,,,,,,,,,,,,,,,,,,
A2,1M,M,Graduate,Other,31-45,-1,-1,-1,-1,-1,-1,3248,3389,6004,39418,162772,-13982,3437,6004,39418,162772,0,538165,0
A3,1M,F,High School,Other,Less than 30,0,-1,-1,-1,-1,0,353351,151818,26948,43530,80811,124590,151818,46200,43530,80811,942,33666,0
A4,100K,F,High School,Single,31-45,4,3,2,2,-2,-2,16681,16082,15477,0,0,0,0,0,0,0,0,0,1
A5,200K,F,Graduate,Single,31-45,2,0,0,0,0,0,90457,92848,95193,97309,100353,102740,3855,3890,3696,4620,4049,3918,1
A6,1M,F,Graduate,Other,31-45,2,2,0,0,0,0,429556,419466,429785,435354,445271,453899,0,20790,16170,17325,16401,17325,0


In [89]:
#read test data set
df_test = pd.read_csv("drive/My Drive/FYP/dataset/credit_card_default_test.csv", index_col=0)
df_test.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC
Client_ID,,,,,,,,,,,,,,,,,,,,,,,
A20170,500K,M,Other,Single,31-45,0,0,0,0,0,0,194188,198510,128316,311201,351790,352206,13710,12816,195701,53246,17256,12991
A16887,200K,F,Other,Single,31-45,0,0,0,0,0,0,87115,89427,91823,94181,97221,100275,3756,3842,3890,4620,4620,4620
A3430,400K,M,Other,Other,Less than 30,-1,-1,2,0,0,-1,2693,9573,8984,11294,9561,993,6930,0,2310,0,2310,40060
A3696,1.5M,M,Graduate,Single,31-45,1,-1,-1,-2,-2,-2,0,5775,5775,0,0,0,5775,5775,0,0,0,0
A20474,100K,F,High School,Other,Less than 30,0,0,-2,-2,-2,-2,85257,0,0,0,0,0,0,0,0,0,0,0


In [74]:
#train data set pre processing

#convet to string balance limit to float
df.Balance_Limit_V1 = (df.Balance_Limit_V1.replace(r'[KMB]+$', '', regex=True).astype(float) * df.Balance_Limit_V1.str.extract(r'[\d\.]+([KMB]+)', expand=False).fillna(1).replace(['K','M', 'B'], [10**3, 10**6, 10**9]).astype(int))
df.head()

#gender education marital age stringto int convert
df.Gender.replace(['M', 'F'], [1, 0], inplace=True)
df.EDUCATION_STATUS.replace(['Graduate', 'High School', 'Other'], [2, 1, 0], inplace=True)

#df.PAY_JULY.replace([-2, -1], [11, 13], inplace=True)
#df.PAY_AUG.replace([-2, -1], [11, 13], inplace=True)
#df.PAY_SEP.replace([-2, -1], [11, 13], inplace=True)
#df.PAY_OCT.replace([-2, -1], [11, 13], inplace=True)
#df.PAY_NOV.replace([-2, -1], [11, 13], inplace=True)
#df.PAY_DEC.replace([-2, -1], [11, 13], inplace=True)

df.MARITAL_STATUS.replace(['Other', 'Single'], [1, 0], inplace=True)
df.AGE.replace(['Less than 30', '31-45', '46-65','More than 65'], [3, 2, 1, 0], inplace=True)

df.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,NEXT_MONTH_DEFAULT
Client_ID,,,,,,,,,,,,,,,,,,,,,,,,
A2,1000000.0,1,2,1,2,-1,-1,-1,-1,-1,-1,3248,3389,6004,39418,162772,-13982,3437,6004,39418,162772,0,538165,0
A3,1000000.0,0,1,1,3,0,-1,-1,-1,-1,0,353351,151818,26948,43530,80811,124590,151818,46200,43530,80811,942,33666,0
A4,100000.0,0,1,0,2,4,3,2,2,-2,-2,16681,16082,15477,0,0,0,0,0,0,0,0,0,1
A5,200000.0,0,2,0,2,2,0,0,0,0,0,90457,92848,95193,97309,100353,102740,3855,3890,3696,4620,4049,3918,1
A6,1000000.0,0,2,1,2,2,2,0,0,0,0,429556,419466,429785,435354,445271,453899,0,20790,16170,17325,16401,17325,0


In [90]:
#test data set pre processing

#convet to string balance limit to float
df_test.Balance_Limit_V1 = (df_test.Balance_Limit_V1.replace(r'[KMB]+$', '', regex=True).astype(float) * df_test.Balance_Limit_V1.str.extract(r'[\d\.]+([KMB]+)', expand=False).fillna(1).replace(['K','M', 'B'], [10**3, 10**6, 10**9]).astype(int))
df_test.head()

#gender education marital age stringto int convert
df_test.Gender.replace(['M', 'F'], [1, 0], inplace=True)
df_test.EDUCATION_STATUS.replace(['Graduate', 'High School', 'Other'], [2, 1, 0], inplace=True)
df_test.MARITAL_STATUS.replace(['Other', 'Single'], [1, 0], inplace=True)
df_test.AGE.replace(['Less than 30', '31-45', '46-65','More than 65'], [3, 2, 1, 0], inplace=True)

df_test.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC
Client_ID,,,,,,,,,,,,,,,,,,,,,,,
A20170,500000.0,1,0,0,2,0,0,0,0,0,0,194188,198510,128316,311201,351790,352206,13710,12816,195701,53246,17256,12991
A16887,200000.0,0,0,0,2,0,0,0,0,0,0,87115,89427,91823,94181,97221,100275,3756,3842,3890,4620,4620,4620
A3430,400000.0,1,0,1,3,-1,-1,2,0,0,-1,2693,9573,8984,11294,9561,993,6930,0,2310,0,2310,40060
A3696,1500000.0,1,2,0,2,1,-1,-1,-2,-2,-2,0,5775,5775,0,0,0,5775,5775,0,0,0,0
A20474,100000.0,0,1,1,3,0,0,-2,-2,-2,-2,85257,0,0,0,0,0,0,0,0,0,0,0


In [75]:
X = df.drop("NEXT_MONTH_DEFAULT",1)   #Feature Matrix
y = df["NEXT_MONTH_DEFAULT"]          #Target Variable
X

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC
Client_ID,,,,,,,,,,,,,,,,,,,,,,,
A2,1000000.0,1,2,1,2,-1,-1,-1,-1,-1,-1,3248,3389,6004,39418,162772,-13982,3437,6004,39418,162772,0,538165
A3,1000000.0,0,1,1,3,0,-1,-1,-1,-1,0,353351,151818,26948,43530,80811,124590,151818,46200,43530,80811,942,33666
A4,100000.0,0,1,0,2,4,3,2,2,-2,-2,16681,16082,15477,0,0,0,0,0,0,0,0,0
A5,200000.0,0,2,0,2,2,0,0,0,0,0,90457,92848,95193,97309,100353,102740,3855,3890,3696,4620,4049,3918
A6,1000000.0,0,2,1,2,2,2,0,0,0,0,429556,419466,429785,435354,445271,453899,0,20790,16170,17325,16401,17325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A29996,1000000.0,0,1,0,2,0,0,0,0,0,0,436470,445403,481323,203289,72157,36914,19635,46200,11557,7039,11550,2310
A29997,400000.0,0,1,1,2,-1,-1,-1,-1,0,0,3888,4223,8090,20741,11989,0,4243,8145,20785,298,0,0
A29998,100000.0,0,0,1,2,4,3,2,-1,0,0,8235,7752,6371,48228,47544,44715,0,0,50820,9702,4620,7161


In [0]:
X = df_test

**Find Gradient of DUE_AMT**

In [93]:
DUE_X = X[["DUE_AMT_JULY", "DUE_AMT_AUG", "DUE_AMT_SEP", "DUE_AMT_OCT", "DUE_AMT_NOV", "DUE_AMT_DEC"]]
DUE_X.head()

,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC
Client_ID,,,,,,
A20170,194188,198510,128316,311201,351790,352206
A16887,87115,89427,91823,94181,97221,100275
A3430,2693,9573,8984,11294,9561,993
A3696,0,5775,5775,0,0,0
A20474,85257,0,0,0,0,0


In [0]:
from statistics import mean

def best_fit_slope(xs,ys):
    m = (((mean(xs)*mean(ys)) - mean(xs*ys)) /
         ((mean(xs)**2) - mean(xs**2)))
    return m

In [95]:
# Create an empty list 
Row_list =[] 
  
# Iterate over each row 
for index, rows in DUE_X.iterrows(): 
    # Create list for the current row 
    my_list =[rows.DUE_AMT_JULY, rows.DUE_AMT_AUG, rows.DUE_AMT_SEP, rows.DUE_AMT_OCT, rows.DUE_AMT_NOV, rows.DUE_AMT_DEC] 

    xs = np.array([1,2,3,4,5,6], dtype=np.float64)
    ys = np.array(my_list, dtype=np.float64)

    # append the list to the final list 
    Row_list.append(best_fit_slope(xs,ys)) 
  
# Print the list 
#print(Row_list) 

X["DUE_AMT_GRD"] = Row_list
X.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,DUE_AMT_GRD
Client_ID,,,,,,,,,,,,,,,,,,,,,,,,
A20170,500000.0,1,0,0,2,0,0,0,0,0,0,194188,198510,128316,311201,351790,352206,13710,12816,195701,53246,17256,12991,40937.571429
A16887,200000.0,0,0,0,2,0,0,0,0,0,0,87115,89427,91823,94181,97221,100275,3756,3842,3890,4620,4620,4620,2615.428571
A3430,400000.0,1,0,1,3,-1,-1,2,0,0,-1,2693,9573,8984,11294,9561,993,6930,0,2310,0,2310,40060,-177.885714
A3696,1500000.0,1,2,0,2,1,-1,-1,-2,-2,-2,0,5775,5775,0,0,0,5775,5775,0,0,0,0,-660.000000
A20474,100000.0,0,1,1,3,0,0,-2,-2,-2,-2,85257,0,0,0,0,0,0,0,0,0,0,0,-12179.571429


**Find Gradient of PAID_AMT**

In [96]:
PAID_X = X[["PAID_AMT_JULY", "PAID_AMT_AUG", "PAID_AMT_SEP", "PAID_AMT_OCT", "PAID_AMT_NOV", "PAID_AMT_DEC"]]
PAID_X.head()

,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC
Client_ID,,,,,,
A20170,13710,12816,195701,53246,17256,12991
A16887,3756,3842,3890,4620,4620,4620
A3430,6930,0,2310,0,2310,40060
A3696,5775,5775,0,0,0,0
A20474,0,0,0,0,0,0


In [97]:
# Create an empty list 
Row_list =[] 
  
# Iterate over each row 
for index, rows in PAID_X.iterrows(): 
    # Create list for the current row 
    my_list =[rows.PAID_AMT_JULY, rows.PAID_AMT_AUG, rows.PAID_AMT_SEP, rows.PAID_AMT_OCT, rows.PAID_AMT_NOV, rows.PAID_AMT_DEC] 

    xs = np.array([1,2,3,4,5,6], dtype=np.float64)
    ys = np.array(my_list, dtype=np.float64)

    # append the list to the final list 
    Row_list.append(best_fit_slope(xs,ys)) 
  
# Print the list 
#print(Row_list) 

X["PAID_AMT_GRD"] = Row_list
X.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,DUE_AMT_GRD,PAID_AMT_GRD
Client_ID,,,,,,,,,,,,,,,,,,,,,,,,,
A20170,500000.0,1,0,0,2,0,0,0,0,0,0,194188,198510,128316,311201,351790,352206,13710,12816,195701,53246,17256,12991,40937.571429,-3792.285714
A16887,200000.0,0,0,0,2,0,0,0,0,0,0,87115,89427,91823,94181,97221,100275,3756,3842,3890,4620,4620,4620,2615.428571,210.971429
A3430,400000.0,1,0,1,3,-1,-1,2,0,0,-1,2693,9573,8984,11294,9561,993,6930,0,2310,0,2310,40060,-177.885714,4864.857143
A3696,1500000.0,1,2,0,2,1,-1,-1,-2,-2,-2,0,5775,5775,0,0,0,5775,5775,0,0,0,0,-660.000000,-1320.000000
A20474,100000.0,0,1,1,3,0,0,-2,-2,-2,-2,85257,0,0,0,0,0,0,0,0,0,0,0,-12179.571429,-0.000000


**Find Gradient of PAY**

In [98]:
PAY_X = X[["PAY_JULY", "PAY_AUG", "PAY_SEP", "PAY_OCT", "PAY_NOV", "PAY_DEC"]]
PAY_X.head()

,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC
Client_ID,,,,,,
A20170,0,0,0,0,0,0
A16887,0,0,0,0,0,0
A3430,-1,-1,2,0,0,-1
A3696,1,-1,-1,-2,-2,-2
A20474,0,0,-2,-2,-2,-2


In [99]:
# Create an empty list 
Row_list =[] 
  
# Iterate over each row 
for index, rows in PAY_X.iterrows(): 
    # Create list for the current row 
    my_list =[rows.PAY_JULY, rows.PAY_AUG, rows.PAY_SEP, rows.PAY_OCT, rows.PAY_NOV, rows.PAY_DEC] 

    #xs = np.array([1,2,3,4,5,6], dtype=np.float64)
    #ys = np.array(my_list, dtype=np.float64)

    # append the list to the final list 
    #Row_list.append(best_fit_slope(xs,ys)) 
    Row_list.append(sum(my_list))
  
# Print the list 
#print(Row_list) 

X["PAY_GRD"] = Row_list
X.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,DUE_AMT_GRD,PAID_AMT_GRD,PAY_GRD
Client_ID,,,,,,,,,,,,,,,,,,,,,,,,,,
A20170,500000.0,1,0,0,2,0,0,0,0,0,0,194188,198510,128316,311201,351790,352206,13710,12816,195701,53246,17256,12991,40937.571429,-3792.285714,0
A16887,200000.0,0,0,0,2,0,0,0,0,0,0,87115,89427,91823,94181,97221,100275,3756,3842,3890,4620,4620,4620,2615.428571,210.971429,0
A3430,400000.0,1,0,1,3,-1,-1,2,0,0,-1,2693,9573,8984,11294,9561,993,6930,0,2310,0,2310,40060,-177.885714,4864.857143,-1
A3696,1500000.0,1,2,0,2,1,-1,-1,-2,-2,-2,0,5775,5775,0,0,0,5775,5775,0,0,0,0,-660.000000,-1320.000000,-7
A20474,100000.0,0,1,1,3,0,0,-2,-2,-2,-2,85257,0,0,0,0,0,0,0,0,0,0,0,-12179.571429,-0.000000,-8


**Find Gradient of PAY AND DUE**

In [100]:
PAY_DUE_X = X[["DUE_AMT_GRD", "PAID_AMT_GRD"]]
PAY_DUE_X.head()

,DUE_AMT_GRD,PAID_AMT_GRD
Client_ID,,
A20170,40937.571429,-3792.285714
A16887,2615.428571,210.971429
A3430,-177.885714,4864.857143
A3696,-660.000000,-1320.000000
A20474,-12179.571429,-0.000000


In [101]:
# Create an empty list 
Row_list =[] 
  
# Iterate over each row 
for index, rows in PAY_DUE_X.iterrows(): 
    # Create list for the current row 
    my_list =[rows.DUE_AMT_GRD, rows.PAID_AMT_GRD] 

    #xs = np.array([1,2,3,4,5,6], dtype=np.float64)
    #ys = np.array(my_list, dtype=np.float64)

    # append the list to the final list 
    #Row_list.append(best_fit_slope(xs,ys)) 
    Row_list.append(sum(my_list)/2)
  
# Print the list 
#print(Row_list) 

X["PAY_DUE_GRD"] = Row_list
X.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,DUE_AMT_GRD,PAID_AMT_GRD,PAY_GRD,PAY_DUE_GRD
Client_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,
A20170,500000.0,1,0,0,2,0,0,0,0,0,0,194188,198510,128316,311201,351790,352206,13710,12816,195701,53246,17256,12991,40937.571429,-3792.285714,0,18572.642857
A16887,200000.0,0,0,0,2,0,0,0,0,0,0,87115,89427,91823,94181,97221,100275,3756,3842,3890,4620,4620,4620,2615.428571,210.971429,0,1413.200000
A3430,400000.0,1,0,1,3,-1,-1,2,0,0,-1,2693,9573,8984,11294,9561,993,6930,0,2310,0,2310,40060,-177.885714,4864.857143,-1,2343.485714
A3696,1500000.0,1,2,0,2,1,-1,-1,-2,-2,-2,0,5775,5775,0,0,0,5775,5775,0,0,0,0,-660.000000,-1320.000000,-7,-990.000000
A20474,100000.0,0,1,1,3,0,0,-2,-2,-2,-2,85257,0,0,0,0,0,0,0,0,0,0,0,-12179.571429,-0.000000,-8,-6089.785714


In [0]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [87]:
# Create a random forest classifier
clf = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1)

# Train the classifier
clf.fit(X_train, y_train)

# Print the name and gini importance of each feature
for feature in zip(X.columns, clf.feature_importances_):
    print(feature)

('Balance_Limit_V1', 0.032074127031244314)
('Gender', 0.011915407404345409)
('EDUCATION_STATUS', 0.017702959886661596)
('MARITAL_STATUS', 0.011199569748810522)
('AGE', 0.019851219675100194)
('PAY_JULY', 0.08199379854247567)
('PAY_AUG', 0.032596447867318884)
('PAY_SEP', 0.021409523742955995)
('PAY_OCT', 0.017163566921870457)
('PAY_NOV', 0.015333174478878582)
('PAY_DEC', 0.01348501911315662)
('DUE_AMT_JULY', 0.04896989225043787)
('DUE_AMT_AUG', 0.0451775384060945)
('DUE_AMT_SEP', 0.04395644299902152)
('DUE_AMT_OCT', 0.043005393230285384)
('DUE_AMT_NOV', 0.04218409225682968)
('DUE_AMT_DEC', 0.04130336509990507)
('PAID_AMT_JULY', 0.04362060064421424)
('PAID_AMT_AUG', 0.04100236143274735)
('PAID_AMT_SEP', 0.04120641821681044)
('PAID_AMT_OCT', 0.03805281384340295)
('PAID_AMT_NOV', 0.03719286108246642)
('PAID_AMT_DEC', 0.03800667134110832)
('DUE_AMT_GRD', 0.05276687664198489)
('PAID_AMT_GRD', 0.05120103148711948)
('PAY_GRD', 0.06455385136144923)
('PAY_DUE_GRD', 0.05307497529330433)


In [0]:
#X['NEXT_MONTH_DEFAULT'] = y
#X.head()
X.to_csv('drive/My Drive/FYP/dataset/credit_card_data_test_pre.csv', encoding='utf-8', mode='w', index=True)